In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import torch
from torch import nn,optim
from sklearn import preprocessing
import torch.nn.functional as func
from torch.utils.data import DataLoader
from torch.autograd import Variable     
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
%matplotlib inline

In [2]:
df_original=pd.read_csv('heart_disease_dataset.csv',sep=';')

In [3]:
df=df_original

In [4]:
df

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [5]:
df.describe()

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [6]:
df.isnull().sum()

age                        0
sex                        0
chest_pain_type            0
resting_blood_pressure     0
cholesterol                0
fasting_blood_sugar        0
rest_ecg                   0
max_heart_rate_achieved    0
exercise_induced_angina    0
st_depression              0
st_slope                   0
num_major_vessels          0
thalassemia                0
target                     0
dtype: int64

In [7]:
X=df.drop(labels='target', axis=1).values
y=df.target.values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [9]:
X_Train = torch.from_numpy(X_train)
y_Train = torch.from_numpy(y_train).type(torch.LongTensor) 

In [10]:
X_Test = torch.from_numpy(X_test)
y_Test = torch.from_numpy(y_test).type(torch.LongTensor) 

In [11]:
train = torch.utils.data.TensorDataset(X_Train,y_Train)
test = torch.utils.data.TensorDataset(X_Test,y_Test)

In [12]:
batch_size = 64

train_loader = DataLoader(train, batch_size = batch_size)
test_loader = DataLoader(test, batch_size = batch_size)

In [13]:
X.shape

(303, 13)

In [25]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(13, 6),
            nn.ReLU(),
            nn.Linear(6, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = Model().to(device)

In [26]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [27]:
count = 0
loss_list = []
iteration_list = []
for epoch in range(100):
    for i, (images, labels) in enumerate(train_loader):
        
        train = Variable(images)
        labels = Variable(labels)
    
        train, labels = train.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(train.float())
        
        loss = loss_fn(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        count += 1
        
        if count % 10 == 0:
            correct = 0
            total = 0
            for images, labels in test_loader: 
                test = Variable(images)
                test=test.to(device)
                outputs = model(test.float())
                
                predicted = torch.max(outputs.data, 1)[1]
                
                total += len(labels)
                
                labels=labels.to(device)
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / float(total)
            
            loss_list.append(loss.data)
            iteration_list.append(count)
        if count % 10 == 0:
            print('Iteration: {}  Loss: {}  Accuracy: {}%'.format(count, loss.data, accuracy))

Iteration: 10  Loss: 0.7060359716415405  Accuracy: 50.81966781616211%
Iteration: 20  Loss: 0.6897463798522949  Accuracy: 50.81966781616211%
Iteration: 30  Loss: 0.7024425268173218  Accuracy: 52.459014892578125%
Iteration: 40  Loss: 0.6887598633766174  Accuracy: 52.459014892578125%
Iteration: 50  Loss: 0.7000229358673096  Accuracy: 54.098358154296875%
Iteration: 60  Loss: 0.6878792643547058  Accuracy: 49.18032455444336%
Iteration: 70  Loss: 0.6980218887329102  Accuracy: 49.18032455444336%
Iteration: 80  Loss: 0.6873850226402283  Accuracy: 49.18032455444336%
Iteration: 90  Loss: 0.6960006356239319  Accuracy: 49.18032455444336%
Iteration: 100  Loss: 0.6868516802787781  Accuracy: 49.18032455444336%
Iteration: 110  Loss: 0.6944153904914856  Accuracy: 49.18032455444336%
Iteration: 120  Loss: 0.6861042976379395  Accuracy: 49.18032455444336%
Iteration: 130  Loss: 0.6930481195449829  Accuracy: 49.18032455444336%
Iteration: 140  Loss: 0.6855911016464233  Accuracy: 49.18032455444336%
Iteration: 1